In [ ]:
import duckdb, pandas as pd, numpy as np
from pathlib import Path
Path("../reports").mkdir(exist_ok=True); Path("../assets").mkdir(exist_ok=True)

con = duckdb.connect(database=':memory:')
orders = pd.read_csv("../data/orders_sample.csv", parse_dates=["order_date"])
spend  = pd.read_csv("../data/marketing_spend.csv")
orders["order_month"] = orders["order_date"].dt.to_period("M").astype(str)

con.register("orders", orders)
con.register("spend", spend)


In [ ]:
con.execute("""
CREATE OR REPLACE TABLE fact_orders AS
SELECT
  order_id,
  order_date,
  order_month,
  customer_id,
  channel,
  revenue
FROM orders;
""")

con.execute("""
CREATE OR REPLACE TABLE first_orders AS
SELECT
  customer_id,
  MIN(order_date) AS first_order_date,
  CAST(strftime(MIN(order_date), '%Y-%m-01') AS DATE) AS cohort_month_date,
  strftime(MIN(order_date), '%Y-%m') AS cohort_month,
  ANY_VALUE(channel) AS acquisition_channel
FROM fact_orders
GROUP BY customer_id;
""")


In [ ]:
con.execute("""
CREATE OR REPLACE TABLE fct AS
SELECT
  f.*,
  fo.cohort_month,
  fo.acquisition_channel,
  DATE_DIFF('month', CAST(strftime(fo.first_order_date, '%Y-%m-01') AS DATE),
            CAST(strftime(f.order_date, '%Y-%m-01') AS DATE)) AS months_since_cohort
FROM fact_orders f
JOIN first_orders fo USING (customer_id);
""")

con.execute("SELECT * FROM fct ORDER BY order_date LIMIT 10;").df()


In [ ]:
cohort_rev = con.execute("""
SELECT
  cohort_month,
  months_since_cohort,
  SUM(revenue) AS revenue
FROM fct
GROUP BY 1,2
ORDER BY 1,2;
""").df()

acq_counts = con.execute("""
SELECT cohort_month, COUNT(DISTINCT customer_id) AS new_customers
FROM first_orders
GROUP BY 1 ORDER BY 1;
""").df()

cohort_rev.head(), acq_counts.head()


In [ ]:
# Assume acquisition channel derives initial cohort via acquisition_channel
# Map spend per month across channels to the cohort month; simple allocation example:
spend["month"] = spend["month"].astype(str)
cac = con.execute("""
SELECT fo.cohort_month,
       SUM(s.spend) FILTER (WHERE s.channel = fo.acquisition_channel) / COUNT(DISTINCT fo.customer_id) AS CAC
FROM first_orders fo
LEFT JOIN spend s ON s.month = fo.cohort_month
GROUP BY fo.cohort_month
ORDER BY fo.cohort_month;
""").df()

# LTV by cohort at each month_since_cohort
ltv = cohort_rev.merge(acq_counts, on="cohort_month", how="left")
ltv["ltv"] = ltv["revenue"] / ltv["new_customers"].replace(0, np.nan)
ltv_pivot = ltv.pivot(index="cohort_month", columns="months_since_cohort", values="ltv").fillna(0).sort_index()
ltv_cum = ltv_pivot.cumsum(axis=1)
ltv_cum.head(), cac.head()


In [ ]:
import matplotlib.pyplot as plt, numpy as np
from xlsxwriter import Workbook

# Create summary table: latest cumulative LTV per cohort vs CAC
latest_month = ltv_cum.columns.max() if len(ltv_cum.columns) else 0
summary = ltv_cum[[latest_month]].rename(columns={latest_month: "LTV_latest"}).reset_index()
summary = summary.merge(cac, on="cohort_month", how="left")
summary["LTV_to_CAC"] = summary["LTV_latest"] / summary["CAC"]

# Excel export
out_xlsx = Path("../reports/cac_ltv_cohorts.xlsx")
with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    ltv_cum.to_excel(writer, sheet_name="LTV_cumulative")
    summary.to_excel(writer, index=False, sheet_name="Summary")
    # autosize
    for name, df in [("LTV_cumulative", ltv_cum.reset_index()), ("Summary", summary)]:
        ws = writer.sheets[name]
        for j, col in enumerate(df.columns):
            width = max(12, min(40, len(str(col))+4, int(df[col].astype(str).str.len().quantile(0.9))+2))
            ws.set_column(j, j, width)

# PNG bar chart for README
plt.figure(figsize=(7,4))
plt.bar(summary["cohort_month"], summary["LTV_latest"], label="LTV")
plt.plot(summary["cohort_month"], summary["CAC"], marker="o", label="CAC")
plt.title("Cohort LTV vs CAC")
plt.xticks(rotation=45, ha="right")
plt.ylabel("USD")
plt.legend()
plt.tight_layout()
out_png = Path("../assets/ltv_vs_cac.png"); plt.savefig(out_png, dpi=160); out_png.as_posix()


In [ ]:
print("../reports/cac_ltv_cohorts.xlsx")
print("../assets/ltv_vs_cac.png")